# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [39]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
cities_df = pd.read_csv('../output_data/cities.csv')
print(cities_df.dtypes)
cities_df.head()

City           object
Lat           float64
Lng           float64
Temp          float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object


,City,Lat,Lng,Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,New Norfolk,-42.7826,147.0587,51.75,54.0,91,16,2.01,AU,1619723501
1,Shache,38.4167,77.2406,68.90,68.9,10,5,7.07,CN,1619723502
2,Busselton,-33.6500,115.3333,50.00,50.0,98,68,14.81,AU,1619723334
3,Honiton,50.7996,-3.1890,46.00,46.0,77,55,8.99,GB,1619723502
4,Águilas,37.4063,-1.5829,63.34,66.0,52,100,7.00,ES,1619723503


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

# Store humidity of cities 
humidity = cities_df["Humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=4)


# Add heat layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [51]:
# A max temperature lower than 80 degrees but higher than 70.
ideal_climate = cities_df[(cities_df["Temp"] < 80) & (cities_df["Temp"] > 70)]

# Wind speed less than 10 mph.
ideal_climate = ideal_climate[ideal_climate["Wind Speed"] < 10]

# Zero cloudiness.
ideal_climate = ideal_climate[ideal_climate["Cloudiness"] == 0]
#ideal_climate

ideal_climate.reset_index(inplace=True, drop=True)
ideal_climate

,City,Lat,Lng,Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Saint-Philippe,-21.3585,55.7679,71.55,77.00,83,0,8.05,RE,1619723339
1,Karratha,-20.7377,116.8463,79.59,79.59,40,0,9.15,AU,1619723515
2,Kiryat Gat,31.6100,34.7642,72.66,77.00,62,0,8.95,IL,1619723520
3,Saint-Pierre,-21.3393,55.4781,72.50,77.00,78,0,6.91,RE,1619723522
4,Rodolfo Sánchez Taboada,31.7167,-116.5667,73.00,73.00,62,0,1.01,MX,1619723529
5,Santa Fe,-31.6333,-60.7000,76.28,77.00,44,0,7.00,AR,1619723527
6,Chiredzi,-21.0500,31.6667,71.76,71.76,47,0,5.66,ZW,1619723540
7,Chui,-33.6971,-53.4616,70.36,70.36,47,0,3.44,UY,1619723542
8,Pithāpuram,17.1167,82.2667,79.50,79.50,66,0,0.85,IN,1619723560
9,La Plata,-34.9215,-57.9545,73.00,73.00,54,0,4.00,AR,1619723575


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [52]:
hotel_df = ideal_climate
hotel_df["Hotel Name"] = ""

#target_coordinates = hotel_df[["Lat", "Lng"]]

for index, row in target_coordinates.iterrows():
        
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row["Lat"]
    lng = row["Lng"]
    try:

        params = {"location": f"{lat}, {lng}",
                  "keyword": "hotel",
                  "radius": 5000,
                  "type": "hotel",
                  "key": g_key,
                  }

        response = requests.get(base_url, params=params).json()
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except:
        hotel_df.loc[index, "Hotel Name"] = "No Hotel Found"
        
hotel_df.reset_index(inplace=True, drop=True)
#print(json.dumps(response, indent=4, sort_keys=True))


In [53]:
hotel_df

,City,Lat,Lng,Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Saint-Philippe,-21.3585,55.7679,71.55,77.00,83,0,8.05,RE,1619723339,Les Embruns Du Baril
1,Karratha,-20.7377,116.8463,79.59,79.59,40,0,9.15,AU,1619723515,Karratha International Hotel
2,Kiryat Gat,31.6100,34.7642,72.66,77.00,62,0,8.95,IL,1619723520,Desert Gat
3,Saint-Pierre,-21.3393,55.4781,72.50,77.00,78,0,6.91,RE,1619723522,Hôtel le Saint-Pierre
4,Rodolfo Sánchez Taboada,31.7167,-116.5667,73.00,73.00,62,0,1.01,MX,1619723529,No Hotel Found
5,Santa Fe,-31.6333,-60.7000,76.28,77.00,44,0,7.00,AR,1619723527,InterTower Hotel
6,Chiredzi,-21.0500,31.6667,71.76,71.76,47,0,5.66,ZW,1619723540,The Nesbitt Arms
7,Chui,-33.6971,-53.4616,70.36,70.36,47,0,3.44,UY,1619723542,Hotel Internacional
8,Pithāpuram,17.1167,82.2667,79.50,79.50,66,0,0.85,IN,1619723560,No Hotel Found
9,La Plata,-34.9215,-57.9545,73.00,73.00,54,0,4.00,AR,1619723575,Hotel Corregidor


In [44]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [47]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))